In [1]:
# We have to do an unsupervised learning.
# But first learn MaLSTM.

from time import time
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import datetime

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Lambda
import keras.backend as K
from keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [2]:
# File paths
TRAIN_CSV = 'train.csv'
TEST_CSV = 'test.csv'
EMBEDDING_FILE = 'GoogleNews-vectors-negative300.bin.gz'

Create embedding matrix

In [3]:
# Load training and test set
train_df = pd.read_csv(TRAIN_CSV)
test_df = pd.read_csv(TEST_CSV)

/anaconda3/envs/keras/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
stops = set(stopwords.words('english'))
print(stops)

{'with', 'yourselves', 'don', "that'll", 'these', 'am', 'doesn', 'its', 'has', 'but', 'when', 'couldn', 'you', 's', 'this', 'up', "wouldn't", 'than', 'why', 'just', 'y', "aren't", 'we', 'very', 'd', 'himself', 'each', "shouldn't", 'most', 'an', 'hasn', 'so', 't', 'while', "didn't", 'mightn', 'itself', 'll', 'her', 'now', 'there', 'hers', 'won', 'whom', "it's", 'are', 'above', 'shouldn', 'their', 'who', 'only', 'his', "should've", "you've", 'haven', 'yours', 'ma', 'down', 'him', 'if', 'had', 'them', 'm', 'was', 'of', 'at', 'yourself', "you'll", 'by', 'needn', 'against', 'some', 'our', 'does', "wasn't", 'in', 'before', 'shan', 'on', 'myself', 'theirs', 'what', 'off', 'such', 'it', "she's", 'herself', "you'd", 'below', 'having', 'where', 'o', 'ain', 'once', "isn't", 'ours', 'any', 'after', 'weren', 'during', 'didn', 'under', 'out', 'do', 'for', 'wouldn', 'between', 'or', 'other', 'me', 'should', 'mustn', 'because', "weren't", 'as', 'were', 'until', 've', 'about', 'have', 'into', 'nor', 'h

In [5]:
def text_to_word_list(text):
    ''' Preprocess and convert texts to a list of words '''
    text = str(text)
    text = text.lower()
    
    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    text = text.split()

    return text

In [8]:
# Prepare embedding
vocabulary = dict()
inverse_vocabulary = ['<unk>']
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

questions_cols = ['question1', 'question2']

In [11]:
# Iterate over the questions only of both training and test datasets
for dataset in [train_df, test_df]:
    for index, row in dataset.iterrows():
        # Iterate through the text of both questions of the row
        for question in questions_cols:
            q2n = []  # q2n -> question numbers representation
            for word in text_to_word_list(row[question]):
                # Check for unwanted words
                if word in stops and word not in word2vec.vocab:
                    continue
                    
                if word not in vocabulary:
                    vocabulary[word] = len(inverse_vocabulary)
                    q2n.append(len(inverse_vocabulary))
                    inverse_vocabulary.append(word)
                else:
                    q2n.append(vocabulary[word])
                    
            # Replace questions as word to question as number representation
            dataset.set_value(index, question, q2n)
            
embedding_dim = 300
embeddings = 1 * np.random.randn(len(vocabulary) + 1, embedding_dim)  # This will be the embedding matrix
embeddings[0] = 0  # So that the padding will be ignored

/anaconda3/envs/keras/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [12]:
# Build the embedding matrix
for word, index in vocabulary.items():
    if word in word2vec.vocab:
        embeddings[index] = word2vec.word_vec(word)
        
del word2vec

In [13]:
train_df

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,"[104495, 98767, 19946, 42566, 6846, 42566, 829...","[104495, 98767, 19946, 42566, 6846, 42566, 829...",0
1,1,3,4,"[104495, 98767, 19946, 60505, 122257, 52565, 1...","[104495, 29696, 121748, 4568, 19946, 99592, 10...",0
2,2,5,6,"[16186, 118033, 125686, 125805, 19946, 126306,...","[16186, 118033, 132297, 126306, 105462, 132923...",0
3,3,7,8,"[124642, 130953, 125686, 132723, 122749, 12817...","[136495, 19946, 130068, 22681, 123188, 129859,...",0
4,4,9,10,"[100283, 134008, 129951, 121628, 129439, 12296...","[100283, 139783, 29696, 122748, 121628, 104907...",0
5,5,11,12,"[134206, 122025, 125686, 130953, 128996, 12292...","[125686, 130953, 137001, 128996, 122924, 14213...",1
6,6,13,14,"[124828, 125686, 132854, 128664]","[104495, 133071, 142748, 124885, 126440, 13285...",0
7,7,15,16,"[16186, 118033, 125686, 105462, 124884, 138916]","[104495, 124828, 125686, 91269, 105462, 133287...",1
8,8,17,18,"[22681, 91269, 122055, 123164, 136458]","[22681, 91269, 122055, 123164, 136458]",0
9,9,19,20,"[146960, 125615, 122025, 118033, 125686, 13573...","[16186, 91269, 125686, 135735, 146960, 128429,...",0


In [19]:
train_df.question1.map(lambda x: len(x)).max()

113

In [14]:
max_seq_length = max(train_df.question1.map(lambda x: len(x)).max(),
                    train_df.question2.map(lambda x: len(x)).max(),
                    test_df.question1.map(lambda x: len(x)).max(),
                    test_df.question2.map(lambda x: len(x)).max())

# Split to train validation
validation_size = 40000
training_size = len(train_df) - validation_size

X = train_df[questions_cols]
Y = train_df['is_duplicate']

X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=validation_size)

# Split to dicts
X_train = {'left': X_train.question1, 'right': X_train.question2}
X_validation = {'left': X_validation.question1, 'right': X_validation.question2}
X_test = {'left': test_df.question1, 'right': test_df.question2}

# Convert labels to their numpy representations
Y_train = Y_train.values
Y_validation = Y_validation.values

# Zero padding
for dataset, side in itertools.product([X_train, X_validation], ['left', 'right']):
    dataset[side] = pad_sequences(dataset[side], maxlen=max_seq_length)

# Make sure everything is ok
assert X_train['left'].shape == X_train['right'].shape
assert len(X_train['left']) == len(Y_train)

In [15]:
X_train

{'left': array([[     0,      0,      0, ..., 171268, 141929, 217153],
        [     0,      0,      0, ..., 136430, 124661, 241316],
        [     0,      0,      0, ..., 257421, 241145, 124642],
        ...,
        [     0,      0,      0, ..., 241699, 153879, 209786],
        [     0,      0,      0, ..., 138362,  98767, 241211],
        [     0,      0,      0, ..., 180827, 132852,  64480]], dtype=int32),
 'right': array([[     0,      0,      0, ..., 171268,  19946, 217153],
        [     0,      0,      0, ..., 241316, 121628, 163782],
        [     0,      0,      0, ..., 269735, 241718, 251074],
        ...,
        [     0,      0,      0, ..., 153879, 209786, 242170],
        [     0,      0,      0, ..., 138362,  98767, 241211],
        [     0,      0,      0, ..., 151739, 244252, 180827]], dtype=int32)}

In [16]:
Y_train

array([0, 0, 0, ..., 0, 1, 0])

Build the model

In [17]:
max_seq_length

213

In [20]:
# Model variables
n_hidden = 50
gradient_clipping_norm = 1.25
batch_size = 64
n_epoch = 25

def exponent_neg_manhattan_distance(left, right):
    '''Helper function for the similarity estimate of the LSTMs outputs'''
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

# The visible layer
left_input = Input(shape=(max_seq_length,), dtype='int32')
right_input = Input(shape=(max_seq_length,), dtype='int32')

embedding_layer = Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_length=max_seq_length, trainable=False)

In [21]:
len(embeddings)

213

In [22]:
embedding_dim

300

In [23]:
[embeddings]

[array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.13964844, -0.00616455,  0.21484375, ...,  0.05712891,
          0.09960938, -0.234375  ],
        [ 0.00704956, -0.07324219,  0.171875  , ...,  0.01123047,
          0.1640625 ,  0.10693359],
        ...,
        [ 0.34087197, -1.89247188, -1.07282769, ..., -0.94661256,
         -0.26634777,  0.48609451],
        [ 0.21038253,  0.0680669 ,  1.06702752, ..., -0.2647256 ,
          0.32227804, -0.64530497],
        [ 0.83758988,  1.57553072,  1.04312691, ...,  1.55361519,
          0.73055449,  1.71418739]])]

In [24]:
max_seq_length

213

In [25]:
# Embedded version of the inputs
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

# Since this is a siamese network, both sides share the same LSTM
shared_lstm = LSTM(n_hidden)

left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)

# Calculateds the distance as defined by the MaLSTM model
malstm_distance = Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]), output_shape=lambda x: (x[0][0], 1))([left_output, right_output])

# Pack it all up into a model
malstm = Model([left_input, right_input], [malstm_distance])

In [ ]:
optimizer = Adadelta(clipnorm=gradient_clipping_norm)

malstm.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])

# Start training
training_start_time = time()

malstm_trained = malstm.fit([X_train['left'], X_train['right']], Y_train, batch_size=batch_size, epochs=n_epoch, validation_data=([X_validation['left'], X_validation['right']], Y_validation))

print("Training time finished.\n{} epochs in {}".format(n_epoch, datetime.timedelta(seconds=time()-training_start_time)))

/anaconda3/envs/keras/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 364290 samples, validate on 40000 samples
Epoch 1/25
136192/364290 [==========>...................] - ETA: 1:14:09 - loss: 0.2896 - acc: 0.6706

In [ ]:
# Plot accuracy
plt.plot(malstm_trained.history['acc'])
plt.plot(malstm_trained.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot loss
plt.plot(malstm_trained.history['loss'])
plt.plot(malstm_trained.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()